### <a name="service">Service Definition

In [ ]:
service = dict([('title', 'Long Term Averages'),
                ('abstract', 'Long term averages of aggregated vegetation indicators, aggregated LST and rainfall estimates'),
                ('id', 'wfp-01-03-04')])

### <a name="parameter">Parameter Definition 

None

### <a name="runtime">Runtime parameter definition

**Input references**

In [ ]:
input_references = 'https://catalog.terradue.com//better-wfp-00007/series/results/search?format=atom&uid=08895f9204851c5ae137d2c6408aa31184fc2b46'


In [ ]:
print(input_references)

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
from osgeo import gdal
from geopandas import GeoDataFrame
import gzip
import cioppy
import shutil
import sys
import numpy as np
import pandas as pd
import math
import re
import os
import re
import cioppy
import requests
sys.path.append('/application/notebook/libexec')
sys.path.append('/workspace/wfp-01-03-04/src/main/app-resources/notebook/libexec')
from aux_functions import calc_average, matrix_sum, write_output_image, get_matrix_list

ciop = cioppy.Cioppy()

#### Calculate Long Term Averages

In [ ]:
#input_references = ['https://catalog.terradue.com//better-wfp-00006/series/results/search?format=atom&uid=30AF5AC6F4D055C80F39FC575F924F48E1200B60','https://catalog.terradue.com//better-wfp-00006/series/results/search?format=atom&uid=A4D8CC5044089E2C67BA1B83E07870E6571FF198','https://catalog.terradue.com//better-wfp-00006/series/results/search?format=atom&uid=9842E3061B537ADC88C2B0B6D1FF2B919C998099']

In [ ]:
def get_info(row):
    search = ciop.search(end_point=row['catalogue_url'], 
                                  params=[],
                                  output_fields='self,enclosure,startdate,enddate,wkt,updated',
                                  model='GeoTime')[0]
    
    series = pd.Series(search)
    
    series['startdate'] = pd.to_datetime(series['startdate'])
    series['enddate'] = pd.to_datetime(series['enddate'])
    return series


In [ ]:
if isinstance(input_references, str):
    input_references = [input_references]

gpd_data = GeoDataFrame(input_references, columns=['catalogue_url'])
#gpd_data = GeoDataFrame(columns=['enclosure', 'start_date', 'end_date', 'product_type', 'aggregation', 'region'])
gpd_data = gpd_data.apply(lambda row: get_info(row), axis=1)


'''for i, enc in enumerate(gpd_data['enclosure'].tolist()):
    print(i)
    print(enc)
    filename = os.path.splitext(os.path.basename(enc))[0]
    file_comp = filename.split('_')
    start_date = file_comp[-2]
    end_date = file_comp[-1]
    product_type = file_comp[0]
    region = file_comp[1]
    aggregation = file_comp[3]'''
    



gpd_data.head(10)

In [ ]:
def get_product(url, dest):
    
    #request_headers = {'X-JFrog-Art-Api': api_key}

    r = requests.get(url)
    
    open(dest, 'wb').write(r.content)
    
    return r.status_code

In [ ]:
def get_metadata(filepath):
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type

In [ ]:
def get_formatted_date(product_reference):
    metadata = ciop.search(end_point=product_reference,
                           params=[],
                           output_fields='identifier,startdate,enddate',
                           model="GeoTime")
    return metadata[0]['startdate'], metadata[0]['enddate']

In [ ]:
def remove_match_duplicates(gdf_match):
    years = gdf_match['startdate'].dt.year.unique()
    for year in years:
        products = gdf_match[gdf_match['startdate'].dt.year == year]
        if len(products.index.values) > 1:
            outdated_indexes = products[products['updated'] != max(products['updated'])].index.values
            gdf_match = gdf_match.drop(outdated_indexes)
    return gdf_match

In [ ]:
def write_properties_file(dataframe, output_name):
    
    title = 'Output %s' % output_name
    first_date = get_formatted_date(dataframe.iloc[0]['self'])[0]
    last_date = get_formatted_date(dataframe.iloc[-1]['self'])[1]
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (dataframe.iloc[0]['wkt']))

In [ ]:
def calc_lta(dataframe):
    file_list = []
    if not os.path.isdir('tmp_data'):
        os.mkdir('tmp_data')
    for enclosure in dataframe['enclosure'].tolist():
        filepath = 'tmp_data/' + os.path.basename(enclosure)
        status = get_product(enclosure, filepath)
        #status = 200 # TEMP
        if status == 200:
            file_list.append(filepath)
    print(file_list)
    if file_list:
        n_years = len(file_list)
        agr_period_matrix = get_matrix_list(file_list)
        print('Aggregations converted to matrices')
        lta = calc_average(agr_period_matrix, n_years)
        projection, geotransform, no_data_value, data_type = get_metadata(file_list[0])
        for file_ in file_list:
            os.remove(file_)
        return lta, projection, geotransform, no_data_value, data_type
    else:
        return None, None, None

In [ ]:
def write_output(lta, period_start_date, period_end_date, product_type, period_N, agr_type, region, projection, geo_transform, image_format, no_data_value, data_type):
    start_day_month = str(period_start_date.month) + '-' + str(period_start_date.day)
    end_day_month = str(period_end_date.month) + '-' + str(period_end_date.day)
    output_name = 'LTA_' + product_type + '_' + region + '_' + str(period_N) + '_' + agr_type + '_' + start_day_month + '_' + end_day_month + '_' + str(period_start_date.year) + '_' + str(period_end_date.year) + '.tif'
    write_output_image(output_name, lta, image_format, data_type, projection, geo_transform, no_data_value=no_data_value)
    return output_name

In [ ]:
gpd_data = gpd_data.sort_values(by='startdate')
while not gpd_data.empty:
    l1 = gpd_data.iloc[0]
    filename = os.path.splitext(os.path.basename(l1['enclosure']))[0].split('_')
    agr = filename[3]
    prod_type = filename[0]
    N_value = filename[2]
    region = filename[1]
    match = gpd_data[(gpd_data['startdate'].dt.day == l1['startdate'].day) & 
                     (gpd_data['startdate'].dt.month == l1['startdate'].month) & 
                     (gpd_data['enddate'].dt.day == l1['enddate'].day) & 
                     (gpd_data['enddate'].dt.month == l1['enddate'].month) &
                     (gpd_data['wkt'] == l1['wkt']) &
                     (gpd_data['enclosure'].str.contains(agr)) &
                     (gpd_data['enclosure'].str.contains(prod_type))]
    
    indexes = match.index.values
    if len(indexes) > 1:
        print(len(match.index.values))
        match = remove_match_duplicates(match)
        print(len(match.index.values))
        lta, projection, geo_transform, no_data_value, data_type = calc_lta(match)
        if lta is not None:
            filename = write_output(lta, match.iloc[0]['startdate'], match.iloc[-1]['enddate'], prod_type, N_value, agr, region, projection, geo_transform, 'GTiff', no_data_value, data_type)
            print(filename)
            write_properties_file(match, filename)
    gpd_data = gpd_data.drop(indexes)

In [ ]:
#first_year = min(gpd_data['start_date'].dt.year.unique())

'''

year_agr = gpd_data[(gpd_data['start_date'].dt.year == first_year)]

print(year_agr)
for i in range(len(year_agr)):
    start_date = year_agr.iloc[i]['start_date']
    end_date = year_agr.iloc[i]['end_date']
    start_day = start_date.day
    start_month = start_date.month
    end_day = end_date.day
    end_month = end_date.month
    product_enclosure = year_agr.iloc[i]['enclosure'].split('_')
    product_type = product_enclosure[0]
    aggregation_type = product_enclosure[3]
    region = product_enclosure[1]
    agr_period = pd_data[(pd_data['start_date'].dt.day == start_day) & (pd_data['start_date'].dt.month == start_month) & 
                         (pd_data['end_date'].dt.day == end_day) & (pd_data['end_date'].dt.month == end_month) & 
                          pd_data['enclosure'].str.startswith(product_type) & pd_data['enclosure'].str.contains(aggregation_type) &
                          pd_data['enclosure'].str.contains(region)]
    agr_years = pd_data['start_date'].dt.year.unique()
    agr_period = agr_period['enclosure'].tolist()
    print(agr_period)
    n_years = len(agr_period)
    period_start = str(min(agr_years)) + '-' + str(start_month) + '-' + str(start_day)
    period_end = str(max(agr_years)) + '-' + str(end_month) + '-' + str(end_day)
    N = int(re.findall('N\d{1,3}', agr_period[0])[0].split('N')[-1])
    try:
        agr_period_matrix = get_matrix_list(agr_period)
        print('Aggregations converted to matrices')
        print(agr_period_matrix[0])
        lta = calc_average(agr_period_matrix, n_years)
        print('LTA calculated successfully')
        print(lta)
        write_output(lta, period_start, period_end, product_type, N, aggregation_type, region, None, None, 'GTiff')
    except Exception as e:
        print('Error calculating Long-term Averages: ' + str(e))'''

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.